In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-bank-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-161251
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-161251


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###


# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
import os
from azureml.train.hyperdrive import uniform, choice, randint

# Specify parameter sampler
ps = RandomParameterSampling( {
        "C": uniform(0.0001, 1000),
        "max_iter": choice(90,125)
    }
)



# Specify a Policy
policy = BanditPolicy(slack_factor = 0.17, evaluation_interval=2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',  
              compute_target=cpu_cluster,
              entry_script='train.py', 
              vm_size = 'STANDARD_D2_V2', 
              vm_priority = 'lowpriority'
            )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_concurrent_runs=4,
                                     max_total_runs=15)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
experiment = exp.submit(config=hyperdrive_config)
RunDetails(experiment).show() 

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
# Get your best run and save the model from that run.
import joblib

best_run = experiment.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']


print('Experiment Run details:',best_run)
print('Best Accuracy after tuning tuning parameters:',best_run_metrics['Accuracy'])
print('Best hyper parameter:',parameter_values)


Experiment Run details: Run(Experiment: udacity-bank-project,
Id: HD_8b3ec607-c375-4ea2-9c09-919ea917dd88_4,
Type: azureml.scriptrun,
Status: Completed)
Best Accuracy from tuning parameters: 0.9185128983308043
Best hyper parameter: ['--C', '693.1147293247191', '--max_iter', '125']


In [8]:
# saving the best model
best_model = best_run.register_model(model_name='LR_best_model.joblib', model_path='./')
best_model

Model(workspace=Workspace.create(name='quick-starts-ws-161251', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-161251'), name=LR_best_model.joblib, id=LR_best_model.joblib:1, version=1, tags={}, properties={})

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
filepath= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path = filepath)

In [10]:
#view sample data
df=ds.to_pandas_dataframe()
df.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [11]:
from train import clean_data

# Use the clean_data function to clean your data.
x,y = clean_data(ds)

In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target = cpu_cluster,
    task="classification",
    primary_metric="accuracy",
    training_data= ds,
    label_column_name='y',
    iterations=30,
    max_concurrent_iterations=4,
    n_cross_validations=5)

In [14]:
# Submit your automl run

### YOUR CODE HERE ###
run=exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cpucluster with default configuration
Running on remote compute: cpucluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-bank-project,AutoML_75c17ce0-fcee-4ab3-97ae-669fc2a577fd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+------------------------------

In [18]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

import joblib
# Retrieve and save your best automl model.
best_model, pipeline = run.get_output()

joblib.dump(pipeline, "automl.joblib")

['automl.joblib']

In [19]:
best_model

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-bank-project,AutoML_75c17ce0-fcee-4ab3-97ae-669fc2a577fd_28,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [20]:
pipeline

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
), random_state=None, reg_alpha=0.5263157894736842, reg_lambda=0.42105263157894735, subsample=0.99))], verbose=False)), ('12', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=719.6856730011514, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('20', Pipeline(memory=None, step